In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64


## Build a scikit-learn model

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  
Y = iris.target
# print(iris.DESCR)

In [3]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y)

GaussianNB(priors=None)

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:

def test_ws_sql_gen(pickle_data):
    WS_URL="http://192.168.88.88:1888/model"
    b64_data = base64.b64encode(pickle_data)
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;


In [6]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
print(lSQL[0:2000])

WITH "RawScores" AS 
(SELECT "ADS"."KEY" AS "KEY", CAST(NULL AS FLOAT(53)) AS "LogProba_0", CAST(NULL AS FLOAT(53)) AS "Proba_0", -1.09861228867 + (-0.5 * -0.267793442091 - (0.5 * ("ADS"."Feature_0" - 5.006) * ("ADS"."Feature_0" - 5.006)) / 0.121764003092) + (-0.5 * -0.112109357731 - (0.5 * ("ADS"."Feature_1" - 3.418) * ("ADS"."Feature_1" - 3.418)) / 0.142276003092) + (-0.5 * -1.68535226038 - (0.5 * ("ADS"."Feature_2" - 1.464) * ("ADS"."Feature_2" - 1.464)) / 0.0295040030924) + (-0.5 * -2.64826613862 - (0.5 * ("ADS"."Feature_3" - 0.244) * ("ADS"."Feature_3" - 0.244)) / 0.0112640030924) AS "Score_0", CAST(NULL AS FLOAT(53)) AS "LogProba_1", CAST(NULL AS FLOAT(53)) AS "Proba_1", -1.09861228867 + (-0.5 * 0.495040594659 - (0.5 * ("ADS"."Feature_0" - 5.936) * ("ADS"."Feature_0" - 5.936)) / 0.261104003092) + (-0.5 * -0.500335172182 - (0.5 * ("ADS"."Feature_1" - 2.77) * ("ADS"."Feature_1" - 2.77)) / 0.0965000030924) + (-0.5 * 0.30725034869 - (0.5 * ("ADS"."Feature_2" - 4.26) * ("ADS"."Feature

## Execute the SQL Code

In [7]:
# save the dataset in a database table

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(X);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['KEY'] = range(lTable.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)


In [8]:
sql_output = pd.read_sql(lSQL , conn);


In [9]:
sql_output.sample(12)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
98,98,None,None,None,4.128729e-30,9.999998e-01,2.313169e-07,-6.765958e+01,-2.313169e-07,-1.527948e+01,1
31,31,None,None,None,1.000000e+00,2.195901e-14,6.176507e-22,-2.198242e-14,-3.144960e+01,-4.883612e+01,0
142,142,None,None,None,5.008456e-151,2.501216e-02,9.749878e-01,-3.460792e+02,-3.688393e+00,-2.533028e-02,2
91,91,None,None,None,3.308727e-100,9.893715e-01,1.062853e-02,-2.290619e+02,-1.068542e-02,-4.544213e+00,1
140,140,None,None,None,5.796680e-220,5.014466e-09,1.000000e+00,-5.048114e+02,-1.911094e+01,-5.014466e-09,2
29,29,None,None,None,1.000000e+00,4.597685e-17,1.258399e-24,0.000000e+00,-3.761839e+01,-5.503220e+01,0
148,148,None,None,None,5.586495e-197,2.460204e-07,9.999998e-01,-4.518889e+02,-1.521785e+01,-2.460205e-07,2
121,121,None,None,None,2.825259e-146,1.376260e-02,9.862374e-01,-3.351388e+02,-4.285801e+00,-1.385818e-02,2
120,120,None,None,None,2.942995e-219,1.171169e-08,1.000000e+00,-5.031867e+02,-1.826268e+01,-1.171169e-08,2
130,130,None,None,None,8.555803e-221,9.062384e-07,9.999991e-01,-5.067247e+02,-1.391396e+01,-9.062389e-07,2


In [10]:
sql_output.Decision.value_counts()

2    50
1    50
0    50
Name: Decision, dtype: int64

## Scikit-learn Prediction

In [11]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
131,131,NaN,NaN,NaN,1.247227e-250,2.967305e-10,1.000000e+00,-575.425351,-2.193820e+01,-2.967306e-10,2
33,33,NaN,NaN,NaN,1.000000e+00,3.197019e-20,1.428817e-26,0.000000,-4.488948e+01,-5.951037e+01,0
29,29,NaN,NaN,NaN,1.000000e+00,4.597685e-17,1.258399e-24,0.000000,-3.761839e+01,-5.503220e+01,0
32,32,NaN,NaN,NaN,1.000000e+00,6.530873e-21,3.118877e-27,0.000000,-4.647775e+01,-6.103232e+01,0
73,73,NaN,NaN,NaN,4.794280e-97,9.981565e-01,1.843481e-03,-221.783330,-1.845182e-03,-6.296100e+00,1
68,68,NaN,NaN,NaN,5.483950e-102,9.946971e-01,5.302892e-03,-233.161854,-5.317002e-03,-5.239503e+00,1
54,54,NaN,NaN,NaN,9.049382e-107,9.524418e-01,4.755819e-02,-244.173908,-4.872626e-02,-3.045801e+00,1
57,57,NaN,NaN,NaN,1.197348e-34,9.999998e-01,2.332069e-07,-78.107784,-2.332069e-07,-1.527134e+01,1
108,108,NaN,NaN,NaN,2.105891e-190,4.929018e-04,9.995071e-01,-436.746429,-7.615201e+00,-4.930233e-04,2
19,19,NaN,NaN,NaN,1.000000e+00,8.944938e-18,7.106919e-25,0.000000,-3.925544e+01,-5.560356e+01,0


## Comparing the SQL and Scikit-learn Predictions

In [12]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [13]:
sql_skl_join.sample(12)

,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
129,129,NaN,NaN,NaN,1.613376e-181,7.018057e-04,0.999298,-416.289573,-7.261854,-7.020521e-04,...,None,None,None,1.613376e-181,7.018057e-04,0.999298,-416.289573,-7.261854,-7.020521e-04,2
113,113,NaN,NaN,NaN,4.082205e-152,1.288071e-02,0.987119,-348.586297,-4.352025,-1.296438e-02,...,None,None,None,4.082205e-152,1.288071e-02,0.987119,-348.586297,-4.352025,-1.296438e-02,2
52,52,NaN,NaN,NaN,1.528218e-122,4.561513e-01,0.543849,-280.491279,-0.784931,-6.090842e-01,...,None,None,None,1.528218e-122,4.561513e-01,0.543849,-280.491279,-0.784931,-6.090842e-01,2
95,95,NaN,NaN,NaN,1.585577e-73,9.998499e-01,0.000150,-167.627763,-0.000150,-8.804041e+00,...,None,None,None,1.585577e-73,9.998499e-01,0.000150,-167.627763,-0.000150,-8.804041e+00,1
117,117,NaN,NaN,NaN,3.753023e-285,1.645749e-12,1.000000,-654.914190,-27.132825,-1.644906e-12,...,None,None,None,3.753023e-285,1.645749e-12,1.000000,-654.914190,-27.132825,-1.645795e-12,2
96,96,NaN,NaN,NaN,1.026621e-77,9.997149e-01,0.000285,-177.272780,-0.000285,-8.162834e+00,...,None,None,None,1.026621e-77,9.997149e-01,0.000285,-177.272780,-0.000285,-8.162834e+00,1
82,82,NaN,NaN,NaN,6.635631e-63,9.999723e-01,0.000028,-143.170407,-0.000028,-1.049580e+01,...,None,None,None,6.635631e-63,9.999723e-01,0.000028,-143.170407,-0.000028,-1.049580e+01,1
81,81,NaN,NaN,NaN,2.140911e-48,9.999987e-01,0.000001,-109.762853,-0.000001,-1.351597e+01,...,None,None,None,2.140911e-48,9.999987e-01,0.000001,-109.762853,-0.000001,-1.351597e+01,1
67,67,NaN,NaN,NaN,2.315345e-63,9.999837e-01,0.000016,-144.223302,-0.000016,-1.102206e+01,...,None,None,None,2.315345e-63,9.999837e-01,0.000016,-144.223302,-0.000016,-1.102206e+01,1
149,149,NaN,NaN,NaN,9.138634e-145,5.600501e-02,0.943995,-331.662328,-2.882314,-5.763442e-02,...,None,None,None,9.138634e-145,5.600501e-02,0.943995,-331.662328,-2.882314,-5.763442e-02,2


In [14]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]


,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
